# 📊 Визуализация и анализ методов оптимизации

**Адаптивный модуль визуализации для всех методов оптимизации**

Этот notebook содержит атомарные блоки визуализации, которые работают с историей любых методов оптимизации.

Каждый блок визуализации независим и может использоваться отдельно.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Параметры отображения
plt.style.use('seaborn-v0_8-darkgrid')
np.set_printoptions(precision=8, suppress=True)

In [2]:
# Параметры целевой функции (должны совпадать с основным notebook)
ALPHA = np.pi / 6
X0_CENTER = 5.0
Y0_CENTER = 2.0
COS_ALPHA = np.cos(ALPHA)
SIN_ALPHA = np.sin(ALPHA)

# Аналитическое решение
X_OPTIMAL = np.array([X0_CENTER, Y0_CENTER, 0.0])
F_OPTIMAL = 0.0

# Параметры визуализации
COLORS_METHODS = {
    'Координатный спуск': '#1f77b4',
    'Наискорейший спуск': '#ff7f0e',
    'Метод Ньютона': '#2ca02c',
    'BFGS': '#d62728',
    'L-BFGS': '#9467bd'
}

COLORS_COORDS = {'x': '#e74c3c', 'y': '#3498db', 'z': '#2ecc71'}

In [3]:
def extract_trajectory_data(history):
    x_coords = []
    y_coords = []
    z_coords = []
    iterations = []

    for i, point in enumerate(history):
        x = point['x']
        x_coords.append(x[0])
        y_coords.append(x[1])
        z_coords.append(x[2])
        iterations.append(i)

    return x_coords, y_coords, z_coords, iterations


def get_color_for_method(method_name):
    return COLORS_METHODS.get(method_name, '#000000')


def format_point(point, decimals=3):
    return f'({point[0]:.{decimals}f}, {point[1]:.{decimals}f}, {point[2]:.{decimals}f})'

In [4]:
optimization_results = {}

In [5]:
def plot_3d_trajectory_single(history, method_name='Метод', figsize=(10, 8)):
    x_coords, y_coords, z_coords, iterations = extract_trajectory_data(history)
    
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')
    
    # Траектория
    ax.plot(x_coords, y_coords, z_coords, 'b-', alpha=0.6, linewidth=2.5, label='Траектория')
    
    # Начальная точка
    start_point = format_point(np.array([x_coords[0], y_coords[0], z_coords[0]]), decimals=1)
    ax.scatter([x_coords[0]], [y_coords[0]], [z_coords[0]], 
              color='green', s=300, marker='o', edgecolors='darkgreen', linewidth=2,
              label=f'Начало: {start_point}', zorder=5)
    
    # Конечная точка (минимум)
    end_point = format_point(np.array([x_coords[-1], y_coords[-1], z_coords[-1]]), decimals=3)
    ax.scatter([x_coords[-1]], [y_coords[-1]], [z_coords[-1]], 
              color='red', s=300, marker='*', edgecolors='darkred', linewidth=2,
              label=f'Найденный минимум: {end_point}', zorder=5)
    
    # Промежуточные точки
    if len(x_coords) > 2:
        ax.scatter(x_coords[1:-1], y_coords[1:-1], z_coords[1:-1],
                  color='blue', s=50, alpha=0.6, label=f'Шаги ({len(history)} итераций)')
    
    # Теоретическое решение (из глобальных переменных)
    ax.scatter([X_OPTIMAL[0]], [X_OPTIMAL[1]], [X_OPTIMAL[2]],
              color='black', s=400, marker='x', linewidth=3,
              label=f'Точное решение ({X_OPTIMAL[0]}, {X_OPTIMAL[1]}, {X_OPTIMAL[2]})', zorder=6)
    
    ax.set_xlabel('x', fontsize=11, fontweight='bold')
    ax.set_ylabel('y', fontsize=11, fontweight='bold')
    ax.set_zlabel('z', fontsize=11, fontweight='bold')
    ax.set_title(f'{method_name}\n({len(history)} итераций)', fontsize=12, fontweight='bold')
    ax.legend(fontsize=10, loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.view_init(elev=20, azim=45)
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ 3D траектория для {method_name} построена')

In [6]:
def plot_3d_trajectory_comparison(methods_data, figsize=(18, 5)):
    n_methods = len(methods_data)
    fig = plt.figure(figsize=figsize)
    
    for idx, (method_name, history) in enumerate(methods_data.items()):
        ax = fig.add_subplot(1, n_methods, idx + 1, projection='3d')
        
        x_coords, y_coords, z_coords, _ = extract_trajectory_data(history)
        
        # Траектория
        color = get_color_for_method(method_name)
        ax.plot(x_coords, y_coords, z_coords, color=color, alpha=0.7, linewidth=2.5, label='Траектория')
        
        # Начальная точка
        ax.scatter([x_coords[0]], [y_coords[0]], [z_coords[0]], 
                  color='green', s=300, marker='o', edgecolors='darkgreen', linewidth=2,
                  label=f'Начало ({x_coords[0]:.1f}, {y_coords[0]:.1f}, {z_coords[0]:.1f})', zorder=5)
        
        # Конечная точка (минимум)
        ax.scatter([x_coords[-1]], [y_coords[-1]], [z_coords[-1]], 
                  color='red', s=300, marker='*', edgecolors='darkred', linewidth=2,
                  label=f'Минимум ({x_coords[-1]:.3f}, {y_coords[-1]:.3f}, {z_coords[-1]:.3f})', zorder=5)
        
        # Промежуточные точки
        if len(x_coords) > 2:
            ax.scatter(x_coords[1:-1], y_coords[1:-1], z_coords[1:-1],
                      color=color, s=50, alpha=0.4, label=f'Шаги ({len(history)} итераций)')
        
        # Теоретическое решение
        ax.scatter([X0_CENTER], [Y0_CENTER], [0], color='black', s=400, marker='x', linewidth=3,
                  label='Решение (5, 2, 0)', zorder=6)
        
        ax.set_xlabel('x', fontsize=10, fontweight='bold')
        ax.set_ylabel('y', fontsize=10, fontweight='bold')
        ax.set_zlabel('z', fontsize=10, fontweight='bold')
        ax.set_title(f'{method_name}\n({len(history)} итераций)', fontsize=11, fontweight='bold')
        ax.legend(fontsize=9, loc='upper right')
        ax.grid(True, alpha=0.3)
        ax.view_init(elev=20, azim=45)
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ 3D траектории для {n_methods} методов построены')

In [7]:
def plot_coordinates_dynamics_single(history, method_name='Метод', figsize=(14, 10)):
    fig, axes = plt.subplots(3, 1, figsize=figsize)
    fig.suptitle(f'Динамика координат: {method_name}', fontsize=14, fontweight='bold')
    
    x_coords, y_coords, z_coords, iterations = extract_trajectory_data(history)
    
    coord_data = [
        ('x', x_coords, X_OPTIMAL[0]),
        ('y', y_coords, X_OPTIMAL[1]),
        ('z', z_coords, X_OPTIMAL[2])
    ]
    
    for ax, (coord_name, coords, analytical_value) in zip(axes, coord_data):
        color = COLORS_COORDS.get(coord_name, '#000000')
        
        ax.plot(iterations, coords, 'o-', linewidth=2.5, markersize=6, 
               color=color, label=f'{coord_name}(k)')
        ax.axhline(y=analytical_value, color='black', linestyle='--', 
                  alpha=0.7, linewidth=2, label=f'Точное значение = {analytical_value}')
        
        ax.set_ylabel(f'{coord_name}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ Графики динамики координат для {method_name} построены')

In [8]:
def plot_coordinates_dynamics_comparison(methods_data, figsize=(16, 12)):
    n_methods = len(methods_data)
    fig, axes = plt.subplots(n_methods, 3, figsize=figsize)
    fig.suptitle('Динамика изменения координат по методам', fontsize=14, fontweight='bold')
    
    if n_methods == 1:
        axes = axes.reshape(1, -1)
    
    analytical_coords = [X0_CENTER, Y0_CENTER, 0.0]
    coord_names = ['x', 'y', 'z']
    
    for row, (method_name, history) in enumerate(methods_data.items()):
        x_coords, y_coords, z_coords, iterations = extract_trajectory_data(history)
        coords_list = [x_coords, y_coords, z_coords]
        
        for col, (coord_name, coords, analytical_val) in enumerate(
            zip(coord_names, coords_list, analytical_coords)):
            
            ax = axes[row, col]
            color = COLORS_COORDS.get(coord_name, '#000000')
            
            ax.plot(iterations, coords, 'o-', linewidth=2.5, markersize=5, 
                   color=color)
            ax.axhline(y=analytical_val, color='black', linestyle='--', 
                      alpha=0.7, linewidth=2, label='Точное решение')
            
            ax.set_ylabel(f'{coord_name}', fontsize=11, fontweight='bold')
            
            if row == n_methods - 1:
                ax.set_xlabel('Номер итерации', fontsize=10, fontweight='bold')
            
            if col == 0:
                ax.text(-0.35, 0.5, method_name, transform=ax.transAxes,
                        fontsize=11, fontweight='bold', va='center', ha='right', 
                        rotation=90)
            
            ax.grid(True, alpha=0.3)
            ax.legend(fontsize=9, loc='best')
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ Графики динамики координат для {n_methods} методов построены')

In [9]:
def plot_function_convergence(methods_data, use_log_scale=True, figsize=(14, 6)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('Сходимость функции', fontsize=14, fontweight='bold')
    
    for method_name, history in methods_data.items():
        f_values = [h['f'] for h in history]
        iterations = list(range(len(f_values)))
        
        color = get_color_for_method(method_name)
        ax1.plot(iterations, f_values, 'o-', linewidth=2.5, markersize=5,
                color=color, label=method_name, alpha=0.7)
    
    ax1.axhline(y=F_OPTIMAL, color='black', linestyle='--', 
               alpha=0.7, linewidth=2, label='Оптимум f* = 0')
    ax1.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax1.set_ylabel('f(x)', fontsize=11, fontweight='bold')
    ax1.set_title('Значение функции', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10, loc='best')
    
    for method_name, history in methods_data.items():
        f_values = [h['f'] for h in history]
        errors = [abs(f - F_OPTIMAL) + 1e-15 for f in f_values]
        iterations = list(range(len(errors)))
        
        color = get_color_for_method(method_name)
        ax2.semilogy(iterations, errors, 'o-', linewidth=2.5, markersize=5,
                    color=color, label=method_name, alpha=0.7)
    
    ax2.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax2.set_ylabel('|f(x) - f*|', fontsize=11, fontweight='bold')
    ax2.set_title('Ошибка (логарифмическая шкала)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, which='both')
    ax2.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ Графики сходимости функции построены')

In [10]:
def plot_gradient_convergence(methods_data, figsize=(14, 6)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('Сходимость по норме градиента', fontsize=14, fontweight='bold')
    
    for method_name, history in methods_data.items():
        grad_norms = [h['grad_norm'] for h in history]
        iterations = list(range(len(grad_norms)))
        
        color = get_color_for_method(method_name)
        ax1.plot(iterations, grad_norms, 'o-', linewidth=2.5, markersize=5,
                color=color, label=method_name, alpha=0.7)
    
    ax1.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax1.set_ylabel('||∇f(x)||', fontsize=11, fontweight='bold')
    ax1.set_title('Норма градиента (линейная шкала)', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10, loc='best')
    
    for method_name, history in methods_data.items():
        grad_norms = [max(h['grad_norm'], 1e-15) for h in history]
        iterations = list(range(len(grad_norms)))
        
        color = get_color_for_method(method_name)
        ax2.semilogy(iterations, grad_norms, 'o-', linewidth=2.5, markersize=5,
                    color=color, label=method_name, alpha=0.7)
    
    ax2.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax2.set_ylabel('||∇f(x)||', fontsize=11, fontweight='bold')
    ax2.set_title('Норма градиента (логарифмическая шкала)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, which='both')
    ax2.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ Графики сходимости по норме градиента построены')

In [11]:
def create_comparison_table(methods_data):
    data = []
    
    for method_name, history in methods_data.items():
        x_final = history[-1]['x']
        f_final = history[-1]['f']
        grad_norm_final = history[-1]['grad_norm']
        
        error_x = np.linalg.norm(x_final - X_OPTIMAL)
        error_f = abs(f_final - F_OPTIMAL)
        
        data.append({
            'Метод': method_name,
            'Итераций': len(history),
            'f(x*)': f'{f_final:.2e}',
            '||∇f(x*)||': f'{grad_norm_final:.2e}',
            'Ошибка ||x-x*||': f'{error_x:.2e}',
            'Ошибка |f-f*|': f'{error_f:.2e}',
            'x*': format_point(x_final),
        })
    
    df = pd.DataFrame(data)
    return df


def print_comparison_table(methods_data):
    df = create_comparison_table(methods_data)
    print('📊 СРАВНЕНИЕ МЕТОДОВ ОПТИМИЗАЦИИ')
    print()
    display(df)
    print(f'\n✓ Таблица сравнения создана для {len(methods_data)} методов')

In [12]:
def plot_distance_to_optimum(methods_data, figsize=(14, 6)):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('Расстояние до оптимума', fontsize=14, fontweight='bold')
    
    for method_name, history in methods_data.items():
        distances = []
        for point in history:
            x = point['x']
            dist = np.linalg.norm(x - X_OPTIMAL)
            distances.append(dist)
        
        iterations = list(range(len(distances)))
        color = get_color_for_method(method_name)
        
        ax1.plot(iterations, distances, 'o-', linewidth=2.5, markersize=5,
                color=color, label=method_name, alpha=0.7)
    
    ax1.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax1.set_ylabel('||x - x*||', fontsize=11, fontweight='bold')
    ax1.set_title('Расстояние (линейная шкала)', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10, loc='best')
    
    for method_name, history in methods_data.items():
        distances = []
        for point in history:
            x = point['x']
            dist = max(np.linalg.norm(x - X_OPTIMAL), 1e-15)
            distances.append(dist)
        
        iterations = list(range(len(distances)))
        color = get_color_for_method(method_name)
        
        ax2.semilogy(iterations, distances, 'o-', linewidth=2.5, markersize=5,
                    color=color, label=method_name, alpha=0.7)
    
    ax2.set_xlabel('Номер итерации', fontsize=11, fontweight='bold')
    ax2.set_ylabel('||x - x*||', fontsize=11, fontweight='bold')
    ax2.set_title('Расстояние (логарифмическая шкала)', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, which='both')
    ax2.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    plt.show()
    
    print(f'✓ Графики расстояния до оптимума построены')

## 📚 ИНСТРУКЦИЯ ПО ИСПОЛЬЗОВАНИЮ

### Из основного notebook (report.ipynb):

```python
# 1. Запустить методы оптимизации и получить историю
cd_method = CoordinateDescentMethod(...)
x_cd, history_cd, _ = cd_method.optimize(x_initial)

sd_method = SteepestDescentMethod(...)
x_sd, history_sd, _ = sd_method.optimize(x_initial)

# 2. Сформировать словарь результатов
methods_data = {
    'Координатный спуск': history_cd,
    'Наискорейший спуск': history_sd
}

# 3. Вызвать функции визуализации
plot_3d_trajectory_comparison(methods_data)
plot_coordinates_dynamics_comparison(methods_data)
plot_function_convergence(methods_data)
plot_gradient_convergence(methods_data)
print_comparison_table(methods_data)
```

### Функции визуализации:

| Функция | Назначение |
|---------|----------|
| `plot_3d_trajectory_single()` | 3D траектория для одного метода |
| `plot_3d_trajectory_comparison()` | 3D траектория для нескольких методов |
| `plot_coordinates_dynamics_single()` | Динамика координат (один метод) |
| `plot_coordinates_dynamics_comparison()` | Динамика координат (несколько методов) |
| `plot_function_convergence()` | Сходимость функции |
| `plot_gradient_convergence()` | Сходимость по норме градиента |
| `plot_distance_to_optimum()` | Расстояние до оптимума |
| `create_comparison_table()` | Таблица сравнения методов |
| `print_comparison_table()` | Вывод таблицы сравнения |

### Атомарность:

- ✓ Каждая функция независима
- ✓ Каждая функция работает с любыми методами
- ✓ Можно использовать любые комбинации функций
- ✓ Адаптивны к количеству методов
